In [104]:
import pandas as pd

In [2]:
infile = '~/Desktop/mid_market_measurement/data/turn_board/unit_turn_non_terminal_stale_work_orders.csv'

In [3]:
df = pd.read_csv(infile)

In [4]:
df.columns

Index(['VHOST', 'TURN_ID', 'SR_ID', 'WO_ID', 'STATUS_CODE', 'WO_STATUS',
       'TERMINAL_STATUS', 'WORK_ORDER_FRESHNESS', 'CREATED_AT', 'UPDATED_AT'],
      dtype='object')

In [98]:
df['CREATED_AT'] = pd.to_datetime(df['CREATED_AT'])
df['UPDATED_AT'] = pd.to_datetime(df['UPDATED_AT'])

In [99]:
# Get the vhosts which average >= 1 work order per unique turn
df2 = df.groupby(by=['VHOST']).aggregate({'TURN_ID': pd.Series.nunique, 'WO_ID': 'count'}).apply(lambda row: row['WO_ID']/row['TURN_ID'], axis=1).reset_index().rename(columns={0: 'WO_TURN_RATIO'})

In [100]:
# For the VHOSTs with >=1 work order per unique turn get the turns
df3 = df[df['VHOST'].isin(df2[df2['WO_TURN_RATIO'] >= 1]['VHOST']) == True]

In [109]:
# For turns with a work order find the rate of terminal work orders
df3[
    (df3.SR_ID.notna()) &
    (df3.CREATED_AT < pd.to_datetime('today').floor('D') + pd.Timedelta(-30, unit='D'))
].groupby(by=['TERMINAL_STATUS']).count()

,VHOST,TURN_ID,SR_ID,WO_ID,STATUS_CODE,WO_STATUS,WORK_ORDER_FRESHNESS,CREATED_AT,UPDATED_AT
TERMINAL_STATUS,,,,,,,,,
NOT_TERMINAL,20702,20702,20702,20702,20702,20702,20702,20702,20702
TERMINAL,133907,133907,133907,133907,133907,133907,0,133907,133907


In [110]:
df3[
    (df3.SR_ID.notna()) &
    (df3.CREATED_AT < pd.to_datetime('today').floor('D') + pd.Timedelta(-30, unit='D'))
].groupby(by=['TERMINAL_STATUS', 'WORK_ORDER_FRESHNESS']).count()

VHOST  TURN_ID  SR_ID  WO_ID  \
TERMINAL_STATUS WORK_ORDER_FRESHNESS                                 
NOT_TERMINAL    FRESH                  3880     3880   3880   3880   
                STALE                 16822    16822  16822  16822   

                                      STATUS_CODE  WO_STATUS  CREATED_AT  \
TERMINAL_STATUS WORK_ORDER_FRESHNESS                                       
NOT_TERMINAL    FRESH                        3880       3880        3880   
                STALE                       16822      16822       16822   

                                      UPDATED_AT  
TERMINAL_STATUS WORK_ORDER_FRESHNESS              
NOT_TERMINAL    FRESH                       3880  
                STALE                      16822